# Predict next month product demands based on historical product demand 

In [1]:
# Just to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Importing libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime

In [2]:
# Importing additional libraries
import seaborn as sns
sns.set()
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from scipy import special
py.offline.init_notebook_mode(connected=True)


# Read data set from manufacturer

In [3]:
df = pd.read_csv('HPD.csv')

In [4]:
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


# 1. Product Code (categorical): One of 2160 products
# 2. Warehouse (categorical): one of four (4) warehouses
# 3. Product category (categorical): one of 33 categories
# 4. Date (date): demand fulfillment date
# 5. Order demanded (integer): target demand value

In [5]:
df.nunique()

Product_Code        2160
Warehouse              4
Product_Category      33
Date                1729
Order_Demand        3828
dtype: int64

In [6]:
df['Order_Demand'].unique()

array(['100 ', '500 ', '50000 ', ..., '(3750)', '(191)', '28250 '],
      dtype=object)

# Make sure our target demand value is float

In [7]:
df['Order_Demand']=df.Order_Demand.str.extract('(\d+)').astype('float32') # converting into float

# Our Dates in YYYY/MM/DD format. For our monthly forecast we need to group the data by month.

In [8]:
df['Date'].unique()

array(['2012/7/27', '2012/1/19', '2012/2/3', ..., '2016/4/17',
       '2016/3/27', '2016/2/21'], dtype=object)

In [9]:
df['month_year'] = pd.to_datetime(df['Date']).dt.to_period('M')
df.head()
temp=df.groupby('month_year')
#start from sept
for i,v in temp:
    print(v)#.Order_Demand.sum())

       Product_Code Warehouse Product_Category      Date  Order_Demand  \
44799  Product_0965    Whse_A     Category_006  2011/1/8           2.0   

      month_year  
44799    2011-01  
       Product_Code Warehouse Product_Category       Date  Order_Demand  \
72252  Product_1724    Whse_A     Category_003  2011/5/31         108.0   

      month_year  
72252    2011-05  
      Product_Code Warehouse Product_Category       Date  Order_Demand  \
8431  Product_1521    Whse_S     Category_019  2011/6/24       85000.0   
8432  Product_1521    Whse_S     Category_019  2011/6/24        7000.0   

     month_year  
8431    2011-06  
8432    2011-06  
       Product_Code Warehouse Product_Category       Date  Order_Demand  \
17249  Product_0608    Whse_C     Category_001  2011/9/27           5.0   
17250  Product_1933    Whse_C     Category_001  2011/9/27          23.0   
72669  Product_1507    Whse_C     Category_019   2011/9/2        1250.0   
74615  Product_0875    Whse_C     Category_023 

        Product_Code Warehouse Product_Category        Date  Order_Demand  \
203773  Product_0979    Whse_J     Category_028  2013/11/26         500.0   
203781  Product_1159    Whse_J     Category_006  2013/11/21      200000.0   
203787  Product_1938    Whse_J     Category_001   2013/11/6           3.0   
203788  Product_1938    Whse_J     Category_001  2013/11/25           2.0   
203804  Product_1157    Whse_J     Category_006  2013/11/25      320000.0   
203806  Product_1157    Whse_J     Category_006  2013/11/11      280000.0   
203814  Product_1361    Whse_J     Category_033  2013/11/22       10000.0   
203864  Product_1512    Whse_J     Category_019  2013/11/18       30000.0   
203933  Product_1970    Whse_J     Category_005   2013/11/6        4000.0   
203934  Product_1970    Whse_J     Category_005   2013/11/6        1000.0   
203940  Product_1970    Whse_J     Category_005   2013/11/6       11000.0   
203941  Product_1970    Whse_J     Category_005   2013/11/7       22000.0   

In [10]:
temp=df.groupby('month_year')
#start from sept
months_year=[]
orders=[]
warehouse=[]
category=[]

for i,v in temp:
    months_year.append(i)
    orders.append(v.Order_Demand.sum())
    warehouse.append(v.Warehouse.unique())
    category.append(v.Product_Category.unique())              
    
d = {'Months': months_year, 'Total_Order': orders, 'Warehouse': warehouse, 'Product_Category': category}
df2 = pd.DataFrame(data=d)

df2.head(68)
df2.Months

0     2011-01
1     2011-05
2     2011-06
3     2011-09
4     2011-10
5     2011-11
6     2011-12
7     2012-01
8     2012-02
9     2012-03
10    2012-04
11    2012-05
12    2012-06
13    2012-07
14    2012-08
15    2012-09
16    2012-10
17    2012-11
18    2012-12
19    2013-01
20    2013-02
21    2013-03
22    2013-04
23    2013-05
24    2013-06
25    2013-07
26    2013-08
27    2013-09
28    2013-10
29    2013-11
       ...   
38    2014-08
39    2014-09
40    2014-10
41    2014-11
42    2014-12
43    2015-01
44    2015-02
45    2015-03
46    2015-04
47    2015-05
48    2015-06
49    2015-07
50    2015-08
51    2015-09
52    2015-10
53    2015-11
54    2015-12
55    2016-01
56    2016-02
57    2016-03
58    2016-04
59    2016-05
60    2016-06
61    2016-07
62    2016-08
63    2016-09
64    2016-10
65    2016-11
66    2016-12
67    2017-01
Name: Months, Length: 68, dtype: period[M]

# We have 68 months of grouped and total order data in new data frame. Continuous period is 2011-09 to 2017-01. Our prediction should start 2017-02. 

In [11]:
df2.head()

,Months,Total_Order,Warehouse,Product_Category
0,2011-01,2.0,[Whse_A],[Category_006]
1,2011-05,108.0,[Whse_A],[Category_003]
2,2011-06,92000.0,[Whse_S],[Category_019]
3,2011-09,6728.0,[Whse_C],"[Category_001, Category_019, Category_023]"
4,2011-10,7.0,"[Whse_C, Whse_S]","[Category_019, Category_011, Category_007]"


# We keep warehouses and product categories as additional features for now.

In [12]:
def WH_Function(val):
    count=[0,0,0,0]
    for item in val:
        if item=="Whse_A":
            count[0]=1
        if item=="Whse_S":
            count[1]=1
        if item=="Whse_C":
            count[2]=1
        if item=="Whse_J":
            count[3]=1
                   
    return count
temp=[]
for i in df2.Warehouse:
    temp.append(WH_Function(i))
df2["Warehouse_Category"]=temp

Whse_A=[]
Whse_S=[]
Whse_C=[]
Whse_J=[]

for i in df2.Warehouse_Category.values:
    Whse_A.append(i[0])
    Whse_S.append(i[1])
    Whse_C.append(i[2])
    Whse_J.append(i[3])
df2["Whse_A"]=Whse_A
df2["Whse_S"]=Whse_S
df2["Whse_C"]=Whse_C
df2["Whse_J"]=Whse_J
df2.head(10)

,Months,Total_Order,Warehouse,Product_Category,Warehouse_Category,Whse_A,Whse_S,Whse_C,Whse_J
0,2011-01,2.0,[Whse_A],[Category_006],"[1, 0, 0, 0]",1,0,0,0
1,2011-05,108.0,[Whse_A],[Category_003],"[1, 0, 0, 0]",1,0,0,0
2,2011-06,92000.0,[Whse_S],[Category_019],"[0, 1, 0, 0]",0,1,0,0
3,2011-09,6728.0,[Whse_C],"[Category_001, Category_019, Category_023]","[0, 0, 1, 0]",0,0,1,0
4,2011-10,7.0,"[Whse_C, Whse_S]","[Category_019, Category_011, Category_007]","[0, 1, 1, 0]",0,1,1,0
5,2011-11,86524.0,"[Whse_C, Whse_A, Whse_J, Whse_S]","[Category_019, Category_028, Category_006, Cat...","[1, 1, 1, 1]",1,1,1,1
6,2011-12,8178525.0,"[Whse_C, Whse_S, Whse_J, Whse_A]","[Category_019, Category_001, Category_006, Cat...","[1, 1, 1, 1]",1,1,1,1
7,2012-01,74331040.0,"[Whse_J, Whse_S, Whse_C, Whse_A]","[Category_028, Category_006, Category_019, Cat...","[1, 1, 1, 1]",1,1,1,1
8,2012-02,86951776.0,"[Whse_J, Whse_C, Whse_S, Whse_A]","[Category_028, Category_006, Category_019, Cat...","[1, 1, 1, 1]",1,1,1,1
9,2012-03,86463216.0,"[Whse_J, Whse_S, Whse_C, Whse_A]","[Category_028, Category_006, Category_019, Cat...","[1, 1, 1, 1]",1,1,1,1


In [13]:
#this function must be shorter!
def PC_Function(val):
    count=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for item in val:
        if item=="Category_001":
            count[0]=1
        if item=="Category_002":
            count[1]=1
        if item=="Category_003":
            count[2]=1
        if item=="Category_004":
            count[3]=1
        if item=="Category_005":
            count[4]=1
        if item=="Category_006":
            count[5]=1
        if item=="Category_007":
            count[6]=1
        if item=="Category_008":
            count[7]=1
        if item=="Category_009":
            count[8]=1
        if item=="Category_010":
            count[9]=1
        if item=="Category_011":
            count[10]=1
        if item=="Category_012":
            count[11]=1    
        if item=="Category_013":
            count[12]=1
        if item=="Category_014":
            count[13]=1
        if item=="Category_015":
            count[14]=1
        if item=="Category_016":
            count[15]=1
        if item=="Category_017":
            count[16]=1
        if item=="Category_018":
            count[17]=1
        if item=="Category_019":
            count[18]=1
        if item=="Category_020":
            count[19]=1
        if item=="Category_021":
            count[20]=1
        if item=="Category_022":
            count[21]=1
        if item=="Category_023":
            count[22]=1
        if item=="Category_024":
            count[23]=1
        if item=="Category_025":
            count[24]=1
        if item=="Category_026":
            count[25]=1
        if item=="Category_027":
            count[26]=1
        if item=="Category_028":
            count[27]=1
        if item=="Category_029":
            count[28]=1
        if item=="Category_030":
            count[29]=1
        if item=="Category_031":
            count[30]=1
        if item=="Category_032":
            count[31]=1
        if item=="Category_033":
            count[32]=1
    return count
temp2=[]
for i in df2.Product_Category:
    temp2.append(PC_Function(i))
df2["Category_label"]=temp2

Category_001=[]
Category_002=[]
Category_003=[]
Category_004=[]
Category_005=[]
Category_006=[]
Category_007=[]
Category_008=[]
Category_009=[]
Category_010=[]
Category_011=[]
Category_012=[]
Category_013=[]
Category_014=[]
Category_015=[]
Category_016=[]
Category_017=[]
Category_018=[]
Category_019=[]
Category_020=[]
Category_021=[]
Category_022=[]
Category_023=[]
Category_024=[]
Category_025=[]
Category_026=[]
Category_027=[]
Category_028=[]
Category_029=[]
Category_030=[]
Category_031=[]
Category_032=[]
Category_033=[]

for i in df2.Category_label.values:
    Category_001.append(i[0])
    Category_002.append(i[1])
    Category_003.append(i[2])
    Category_004.append(i[3])
    Category_005.append(i[4])
    Category_006.append(i[5])
    Category_007.append(i[6])
    Category_008.append(i[7])
    Category_009.append(i[8])
    Category_010.append(i[9])
    Category_011.append(i[10])
    Category_012.append(i[11])
    Category_013.append(i[12])
    Category_014.append(i[13])
    Category_015.append(i[14])
    Category_016.append(i[15])
    Category_017.append(i[16])
    Category_018.append(i[17])
    Category_019.append(i[18])
    Category_020.append(i[19])
    Category_021.append(i[20])
    Category_022.append(i[21])
    Category_023.append(i[22])
    Category_024.append(i[23])
    Category_025.append(i[24])
    Category_026.append(i[25])
    Category_027.append(i[26])
    Category_028.append(i[27])
    Category_029.append(i[28])
    Category_030.append(i[29])
    Category_031.append(i[30])
    Category_032.append(i[31])
    Category_033.append(i[32])
    
df2["Category_001"]=Category_001
df2["Category_002"]=Category_002
df2["Category_003"]=Category_003
df2["Category_004"]=Category_004
df2["Category_005"]=Category_005
df2["Category_006"]=Category_006
df2["Category_007"]=Category_007
df2["Category_008"]=Category_008
df2["Category_009"]=Category_009
df2["Category_010"]=Category_010
df2["Category_011"]=Category_011
df2["Category_012"]=Category_012
df2["Category_013"]=Category_013
df2["Category_014"]=Category_014
df2["Category_015"]=Category_015
df2["Category_016"]=Category_016
df2["Category_017"]=Category_017
df2["Category_018"]=Category_018
df2["Category_019"]=Category_019
df2["Category_020"]=Category_020
df2["Category_021"]=Category_021
df2["Category_022"]=Category_022
df2["Category_023"]=Category_023
df2["Category_024"]=Category_024
df2["Category_025"]=Category_025
df2["Category_026"]=Category_026
df2["Category_027"]=Category_027
df2["Category_028"]=Category_028
df2["Category_029"]=Category_029
df2["Category_030"]=Category_030
df2["Category_031"]=Category_031
df2["Category_032"]=Category_032
df2["Category_033"]=Category_033

df2.head(10)

,Months,Total_Order,Warehouse,Product_Category,Warehouse_Category,Whse_A,Whse_S,Whse_C,Whse_J,Category_label,...,Category_024,Category_025,Category_026,Category_027,Category_028,Category_029,Category_030,Category_031,Category_032,Category_033
0,2011-01,2.0,[Whse_A],[Category_006],"[1, 0, 0, 0]",1,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
1,2011-05,108.0,[Whse_A],[Category_003],"[1, 0, 0, 0]",1,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
2,2011-06,92000.0,[Whse_S],[Category_019],"[0, 1, 0, 0]",0,1,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
3,2011-09,6728.0,[Whse_C],"[Category_001, Category_019, Category_023]","[0, 0, 1, 0]",0,0,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
4,2011-10,7.0,"[Whse_C, Whse_S]","[Category_019, Category_011, Category_007]","[0, 1, 1, 0]",0,1,1,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, ...",...,0,0,0,0,0,0,0,0,0,0
5,2011-11,86524.0,"[Whse_C, Whse_A, Whse_J, Whse_S]","[Category_019, Category_028, Category_006, Cat...","[1, 1, 1, 1]",1,1,1,1,"[0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...",...,0,0,0,0,1,0,0,0,0,0
6,2011-12,8178525.0,"[Whse_C, Whse_S, Whse_J, Whse_A]","[Category_019, Category_001, Category_006, Cat...","[1, 1, 1, 1]",1,1,1,1,"[1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...",...,1,0,1,0,1,0,1,0,1,0
7,2012-01,74331040.0,"[Whse_J, Whse_S, Whse_C, Whse_A]","[Category_028, Category_006, Category_019, Cat...","[1, 1, 1, 1]",1,1,1,1,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...",...,1,0,1,0,1,1,1,1,1,1
8,2012-02,86951776.0,"[Whse_J, Whse_C, Whse_S, Whse_A]","[Category_028, Category_006, Category_019, Cat...","[1, 1, 1, 1]",1,1,1,1,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...",...,1,1,1,0,1,1,1,1,1,1
9,2012-03,86463216.0,"[Whse_J, Whse_S, Whse_C, Whse_A]","[Category_028, Category_006, Category_019, Cat...","[1, 1, 1, 1]",1,1,1,1,"[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...",...,1,1,1,0,1,1,1,1,1,1


# Create new data frame with categorical features as columns

In [14]:
df3 = df2[['Months', 'Whse_A', 'Whse_S', 'Whse_C', 'Whse_J', 'Category_001', 'Category_002', 'Category_003', 
'Category_004', 'Category_005', 'Category_006', 'Category_007', 'Category_008', 'Category_009', 'Category_010', 
'Category_011', 'Category_012', 'Category_013', 'Category_014', 'Category_015', 'Category_016', 'Category_017', 
'Category_018', 'Category_019', 'Category_020', 'Category_021', 'Category_022', 'Category_023', 'Category_024',
'Category_025', 'Category_026', 'Category_027', 'Category_028', 'Category_029', 'Category_030', 'Category_031', 
'Category_032', 'Category_033', 'Total_Order']]

In [15]:
df3.head(10)

,Months,Whse_A,Whse_S,Whse_C,Whse_J,Category_001,Category_002,Category_003,Category_004,Category_005,...,Category_025,Category_026,Category_027,Category_028,Category_029,Category_030,Category_031,Category_032,Category_033,Total_Order
0,2011-01,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.0
1,2011-05,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,108.0
2,2011-06,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,92000.0
3,2011-09,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6728.0
4,2011-10,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.0
5,2011-11,1,1,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,86524.0
6,2011-12,1,1,1,1,1,0,1,1,1,...,0,1,0,1,0,1,0,1,0,8178525.0
7,2012-01,1,1,1,1,1,0,1,1,1,...,0,1,0,1,1,1,1,1,1,74331040.0
8,2012-02,1,1,1,1,1,0,1,1,1,...,1,1,0,1,1,1,1,1,1,86951776.0
9,2012-03,1,1,1,1,1,0,1,1,1,...,1,1,0,1,1,1,1,1,1,86463216.0


# Use autoregression (AR) method to forecast. 
# It helps to model the next step in the sequence as a linear function of the observations at prior time steps.

# Autoregression

In [16]:
# AR example
from statsmodels.tsa.ar_model import AR

# fit model
model = AR(df3.Total_Order.values)
model_fit = model.fit()

# make prediction
yhat = model_fit.predict(len(df3.Total_Order.values), len(df3.Total_Order.values))
print(yhat)

[60928419.4414116]


In [17]:
# ARIMA example
from statsmodels.tsa.arima_model import ARIMA

# fit model
model = ARIMA(df3.Total_Order.values)
model_fit = model.fit(disp=False)
# make prediction
yhat = model_fit.predict(len(df3.Total_Order.values), len(df3.Total_Order.values), typ='levels')
print(yhat)

TypeError: __new__() missing 1 required positional argument: 'order'

# Forecasting with ARIMA using TIMESERIES

In [18]:
from statsmodels.tsa.seasonal import seasonal_decompose

temp={}
a=df["Date"]
temp["Order"]= df['Order_Demand']
data=pd.DataFrame(temp)
data.index = pd.to_datetime(a)

In [19]:
data.head()

,Order
Date,
2012-07-27,100.0
2012-01-19,500.0
2012-02-03,500.0
2012-02-09,500.0
2012-03-02,500.0


In [20]:
#temp={}
#a=df3["Months"]
#temp["Order"]= df3['Total_Order']
#data=pd.DataFrame(temp)
#data.index = pd.to_datetime(a)

In [21]:
#data["Date"]=pd.date_range(data.Date)

In [22]:
#for value in data.Order.values:
#    if value==0:
#        print("hi")

In [23]:
from plotly.plotly import plot_mpl
from statsmodels.tsa.seasonal import seasonal_decompose
#result = seasonal_decompose(data, model=’additive’)
result = seasonal_decompose(data, freq=30)

In [ ]:
from plotly.offline import download_plotlyjs
from plotly.offline import init_notebook_mode
from plotly.offline import plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

fig = result.plot()
plot_mpl(fig)

In [ ]:
from pmdarima.arima import auto_arima
stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
print(stepwise_model.aic())

In [ ]:
train = data.loc['2011-01-01':'2016-12-01']
test = data.loc['2017-01-01':]

In [ ]:
stepwise_model.fit(train)

In [ ]:
future_forecast = stepwise_model.predict(n_periods=68) # period might be day/month

In [ ]:
#conda create -n pmdarima --yes python=3.7 scipy numpy scikit-learn pandas statsmodels